# 案例 1.1: 承压管道与阀门仿真

本案例旨在演示承压管道和阀门的物理动态。我们将仿真在一个简单的重力供水系统中，随着阀门开度的不同，流量和水头损失是如何变化的。

### 1. 导入库并加载配置

In [ ]:
import json
from swp.simulation_identification.physical_objects.pipe import Pipe
from swp.simulation_identification.physical_objects.valve import Valve

CONFIG_PATH = 'examples/pipe_and_valve_simulation.json'

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

### 2. 定义仿真函数

此函数将根据配置初始化组件，并模拟不同阀门开度下的系统稳态。

In [ ]:
def run_pipe_valve_simulation(cfg):
    # 1. 根据配置初始化组件
    pipe = Pipe(name="main_pipe", initial_state={}, parameters=cfg['pipe_params'])
    valve = Valve(name="control_valve", initial_state={'opening': 100.0}, parameters=cfg['valve_params'])

    # 2. 定义系统边界条件
    bc = cfg['boundary_conditions']
    upstream_head = bc['upstream_head']
    downstream_head_final = bc['downstream_head']
    total_head_diff = upstream_head - downstream_head_final

    print("系统边界条件:")
    print(f"  - 上游水头: {upstream_head:.2f} m")
    print(f"  - 下游水头: {downstream_head_final:.2f} m")
    print(f"  - 总可用水头: {total_head_diff:.2f} m\n")

    # 3. 模拟不同阀门开度的场景
    openings = cfg['simulation_scenarios']['valve_openings']
    solver_iterations = cfg['solver_settings']['iterations']
    
    for opening in openings:
        # 为当前场景设定阀门开度
        valve.step({'control_signal': opening}, dt=1)

        # 4. 使用二分法寻找平衡流量和中间水头
        # 这是手动寻找管流=阀流（Q_pipe = Q_valve）平衡点的方法
        low_h_guess = downstream_head_final
        high_h_guess = upstream_head
        intermediate_head = (low_h_guess + high_h_guess) / 2

        for _ in range(solver_iterations):
            pipe_action = {'upstream_head': upstream_head, 'downstream_head': intermediate_head}
            pipe.step(pipe_action, dt=1)
            q_pipe = pipe.get_state()['outflow']

            valve_action = {'upstream_head': intermediate_head, 'downstream_head': downstream_head_final}
            valve.step(valve_action, dt=1)
            q_valve = valve.get_state()['outflow']

            if q_pipe > q_valve:
                low_h_guess = intermediate_head
            else:
                high_h_guess = intermediate_head
            intermediate_head = (low_h_guess + high_h_guess) / 2

        # 5. 获取收敛后的最终稳态结果
        final_q = pipe.get_state()['outflow']
        pipe_head_loss = upstream_head - intermediate_head
        valve_head_loss = intermediate_head - downstream_head_final

        print(f"--- 阀门开度: {opening}% ---")
        print(f"  平衡流量: {final_q:.4f} m^3/s")
        print(f"  中间水头 (管道与阀门之间): {intermediate_head:.2f} m")
        print(f"  管道水头损失 (摩擦): {pipe_head_loss:.2f} m")
        print(f"  阀门水头损失 (节流): {valve_head_loss:.2f} m")
        print(f"  总计算水头损失: {pipe_head_loss + valve_head_loss:.2f} m (应与总可用水头相等)")
        print("-" * 25)

### 3. 运行仿真

In [ ]:
run_pipe_valve_simulation(config)